# Лабораторная работа 3. Pandas, метод ближайших соседей и решающие деревья.

Name:Beknur Kattabekov

Student ID: 16BD02079

Email:kattabekovbeknur@gmail.com

In [2]:
import numpy as np
import pandas as pd

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных) Answer:Security delay
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.Answer: Mean - 7.263870e+02, min - 1.100000e+01, max- 4.962000e+03.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни? on Monday of 1st January, distances 7.263870e+02, 5.621018e+02, 1.100000e+01, 3.250000e+02, 5.810000e+02, 9.540000e+02, 4.962000e+03
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится? 7.009728e+06, SouthWest Airlines co, Michigan, Florida, Georgia
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя? Hawaiian Airlines Inc
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`). NorthWest Airlines

In [3]:
data = pd.read_csv("2008.csv.bz2")

In [4]:
data.describe(include = 'all')

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,7009728.0,7.009728e+06,7.009728e+06,7.009728e+06,6.873482e+06,7.009728e+06,6.858079e+06,7.009728e+06,7009728,7.009728e+06,...,6.858079e+06,6.872670e+06,7.009728e+06,137434,7.009728e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,...,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WN,NaN,...,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1201754,NaN,...,NaN,NaN,NaN,54904,NaN,NaN,NaN,NaN,NaN,NaN
mean,2008.0,6.375130e+00,1.572801e+01,3.924182e+00,1.333830e+03,1.326086e+03,1.481258e+03,1.494801e+03,NaN,2.224200e+03,...,6.860852e+00,1.645305e+01,1.960618e-02,NaN,2.463006e-03,1.577206e+01,3.039031e+00,1.716462e+01,7.497434e-02,2.077098e+01
std,0.0,3.406737e+00,8.797068e+00,1.988259e+00,4.780689e+02,4.642509e+02,5.052251e+02,4.826728e+02,NaN,1.961716e+03,...,4.933649e+00,1.133280e+01,1.386426e-01,NaN,4.956753e-02,4.009912e+01,1.950287e+01,3.189495e+01,1.837940e+00,3.925964e+01
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,NaN,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2008.0,3.000000e+00,8.000000e+00,2.000000e+00,9.280000e+02,9.250000e+02,1.107000e+03,1.115000e+03,NaN,6.220000e+02,...,4.000000e+00,1.000000e+01,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2008.0,6.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.320000e+03,1.512000e+03,1.517000e+03,NaN,1.571000e+03,...,6.000000e+00,1.400000e+01,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
75%,2008.0,9.000000e+00,2.300000e+01,6.000000e+00,1.728000e+03,1.715000e+03,1.909000e+03,1.907000e+03,NaN,3.518000e+03,...,8.000000e+00,1.900000e+01,0.000000e+00,NaN,0.000000e+00,1.600000e+01,0.000000e+00,2.100000e+01,0.000000e+00,2.600000e+01


In [ ]:
grouped = data.groupby('FlightNum')
grouped.filter(lambda x: x['FlightNum'].mean() > 1000)


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,...,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0
11,2008,1,3,4,1644.0,1510,1845.0,1725,WN,1333,...,6.0,8.0,0,NaN,0,8.0,0.0,0.0,0.0,72.0
13,2008,1,3,4,715.0,715,720.0,710,WN,1016,...,7.0,21.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
14,2008,1,3,4,1702.0,1700,1651.0,1655,WN,1827,...,4.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
15,2008,1,3,4,1029.0,1020,1021.0,1010,WN,2272,...,6.0,9.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
17,2008,1,3,4,754.0,745,940.0,955,WN,1144,...,5.0,16.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
23,2008,1,3,4,1039.0,1030,1133.0,1140,WN,1244,...,2.0,5.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
24,2008,1,3,4,801.0,800,902.0,910,WN,2101,...,3.0,5.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [10]:
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [3]:
data = pd.read_csv('amazon.csv')

In [5]:
data.describe()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
count,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000,32769.000000
mean,0.942110,42923.916171,25988.957979,116952.627788,118301.823156,118912.779914,125916.152644,170178.369648,183703.408893,119789.430132
std,0.233539,34173.892702,35928.031650,10875.563591,4551.588572,18961.322917,31036.465825,69509.462130,100488.407413,5784.275516
min,0.000000,0.000000,25.000000,4292.000000,23779.000000,4674.000000,117879.000000,4673.000000,3130.000000,117880.000000
25%,1.000000,20299.000000,4566.000000,117961.000000,118102.000000,118395.000000,118274.000000,117906.000000,118363.000000,118232.000000
50%,1.000000,35376.000000,13545.000000,117961.000000,118300.000000,118921.000000,118568.000000,128696.000000,119006.000000,118570.000000
75%,1.000000,74189.000000,42034.000000,117961.000000,118386.000000,120535.000000,120006.000000,235280.000000,290919.000000,119348.000000
max,1.000000,312153.000000,311696.000000,311178.000000,286791.000000,286792.000000,311867.000000,311867.000000,308574.000000,270691.000000


In [6]:
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [25]:
data.shape 

(32769, 10)

In [7]:
# доля положительных примеров
data.ACTION.mean()

0.9421099209618847

In [8]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

In [18]:
from sklearn.neighbors import DistanceMetric
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
model = KNeighborsClassifier(n_neighbors = 10, metric = 'euclidean')
model.fit(X_train,y_train)
y_true, y_pred = y_test, model.predict(X_test)
roc_auc_score(y_true, y_pred)

0.5176954413023741

In [19]:
model = KNeighborsClassifier(n_neighbors = 10, metric = 'cosine')
model.fit(X_train,y_train)
y_true, y_pred = y_test, model.predict(X_test)
roc_auc_score(y_true, y_pred)

0.5184162628556395

In [20]:
model = KNeighborsClassifier(n_neighbors = 10, metric = 'manhattan')
model.fit(X_train,y_train)
y_true, y_pred = y_test, model.predict(X_test)
roc_auc_score(y_true, y_pred)

0.5239873993707233

In [5]:
import sklearn
print (sklearn.__version__)

0.20.0


#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

In [40]:
model = KNeighborsClassifier(n_neighbors = 1, weights = 'distance', metric = 'euclidean')
model.fit(X_train,y_train)
y_true, y_pred = y_test, model.predict(X_test)
roc_auc_score(y_true, y_pred)

0.6182969526015349

In [43]:
model = KNeighborsClassifier(n_neighbors = 1, weights = 'distance', metric = 'cosine')
model.fit(X_train,y_train)
y_true, y_pred = y_test, model.predict(X_test)
roc_auc_score(y_true, y_pred)

0.6178470456559852

In [44]:
model = KNeighborsClassifier(n_neighbors = 1, weights = 'distance', metric = 'manhattan')
model.fit(X_train,y_train)
y_true, y_pred = y_test, model.predict(X_test)
roc_auc_score(y_true, y_pred)

0.629726583812623

In [72]:
X_train.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
1728,40867,8150,117961,118386,119954,118054,118054,117887,118055
22856,38587,14551,5110,117954,117895,117899,117897,19721,117900
27691,3853,51301,118290,118291,121108,126869,118302,118295,126870
8421,45337,49423,117961,118052,123144,118396,269406,118398,118399
5013,40919,87700,118315,118463,118464,118465,162182,118467,118468


In [79]:
model.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='distance')

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [73]:
X_train.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
1728,40867,8150,117961,118386,119954,118054,118054,117887,118055
22856,38587,14551,5110,117954,117895,117899,117897,19721,117900
27691,3853,51301,118290,118291,121108,126869,118302,118295,126870
8421,45337,49423,117961,118052,123144,118396,269406,118398,118399
5013,40919,87700,118315,118463,118464,118465,162182,118467,118468


In [75]:
# data.groupby(['ACTION','RESOURCE']).count()
X_train = X_train.groupby(['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']).size().reset_index(name='count')

In [76]:
X_train.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,count
0,0,33,117961,118413,119968,118784,117906,290919,118786,1
1,0,4417,117961,118413,119968,118841,118842,118643,118843,1
2,0,4417,117961,118413,119968,118841,123378,118643,118843,1
3,0,4468,117961,118413,119968,118784,118785,290919,118786,1
4,0,4468,117961,118413,119968,118784,147114,290919,118786,1


#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

Здесь вы можете поделиться своими мыслями о задании.

In [ ]:
Число successes объектов первого класса ( y=1 ) в обучающей выборке с таким же значением признака. В тренировочной части не имеется деление успеха = 1

А здесь вставьте смешную картинку.

А здесь посоветуйте преподавателям хороший фильм или сериал.

In [ ]:
who am i